# FIFA WORLD CUP 2022 SENTIMENT ANALYSIS

## Introduction
This is a twitter sentiment analysis project of the 2022 FIFA WORLD CUP in Qatar. To find out what football fans think about the FIFA World Cup 2022, I'll conduct a Twitter sentiment analysis using some World Cup hashtags. The tweet is taken from diffreent days of the world cup and combined into one master dataset. 

## Importing Useful libraries

In [158]:
#Import the useful libraries for project environment
import snscrape.modules.twitter as sntwitter
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob # to return all the CSV files’ list located within the path
import time
import preprocessor as p # To clean the tweet fro hastags, usernames and emojis
import re

##  Web Scrapping

### Code

In [122]:
#Scraping relevant tweets using snscrape from twitter

query = '((#fifaworldcup2022 OR #fifaworldcup OR #worldcup2022) lang:en until:2022-12-21 since:2022-11-14)'
tweets = []
limit = 50

for tweet in sntwitter.TwitterSearchScraper(query).get_items():
    if len(tweets) == limit:
        break
    else:
        tweets.append([tweet.date, tweet.user.username, tweet.user.location, 
                tweet.rawContent, tweet.retweetCount, tweet.likeCount])

#Save the data to a dataframe

df = pd.DataFrame(tweets, columns=["Date", "Username", "Locations", "Content", "Retweets", "Favorites"])



### Test

In [123]:
# Check the dataframe
df.head()

,Date,Username,Locations,Content,Retweets,Favorites
0,2022-12-20 23:59:08+00:00,knew_channel,Hong Kong,No one to talk or make hype on this such a sha...,0,0
1,2022-12-20 23:58:15+00:00,christo_comedy,"Abia, Nigeria",Shout out verse fake life 😩\n-\nBirthday Bless...,0,1
2,2022-12-20 23:57:55+00:00,beINSPORTSUSA,United States,This is how Leo Messi arrived at his home in R...,29,172
3,2022-12-20 23:57:51+00:00,StatsDataFacts,"London, England",Countries with the most FIFA World Cup titles ...,0,2
4,2022-12-20 23:57:51+00:00,CharlesTheSwift,New York,Will @KMbappe hairline survive until the next ...,0,0


In [76]:
#Save the data to csv file

df.to_csv("Dec_.csv", index=False)

### Define  
Merge the csv files into one file

### Code

In [128]:
### Check the files in the directory
os.chdir(r"C:\Users\user\Documents\DA Projects\Python\FIFA_CSV")
csv_files = glob.glob( '*.csv' )
csv_files

['FIFA_World_Cup_2022.csv',
 'FIFA_WORLD_CUP_2022_Dec_10.csv',
 'FIFA_WORLD_CUP_2022_Dec_11.csv',
 'FIFA_WORLD_CUP_2022_Dec_13_14.csv',
 'FIFA_WORLD_CUP_2022_Dec_15_17.csv',
 'FIFA_WORLD_CUP_2022_Dec_18.csv',
 'FIFA_WORLD_CUP_2022_Dec_19.csv',
 'FIFA_WORLD_CUP_2022_Dec_20.csv',
 'FIFA_WORLD_CUP_2022_Nov_14-17.csv',
 'FIFA_WORLD_CUP_2022_Nov_18-19.csv',
 'FIFA_WORLD_CUP_2022_Nov_21_20.csv']

In [129]:
path = r"C:\Users\user\Documents\DA Projects\Python\FIFA_CSV"
all_csv_files = glob.glob(path + "/*.csv")

tweets = []

# Convert each csv to a dataframe

for filename in all_csv_files:
    df = pd.read_csv(filename, index_col = None, header = 0) 
    tweets.append(df)
    
# Merge all dataframes

FIFA_df = pd.concat(tweets, axis=0, ignore_index = True) 

FIFA_df.head()

,Date,Username,Locations,Content,Retweets,Favorites
0,2022-12-10 23:59:57+00:00,OSUBlockie,"Columbus, OH",I just earned the 'World Pint (2022)' badge on...,0,0
1,2022-12-10 23:59:54+00:00,standardsport,"London, England",France 🆚 Morocco\n\n📆 Date\n⏰ Kick-off time\n📺...,0,5
2,2022-12-10 23:59:52+00:00,MpatrioticM,ⵜⴰⴳⵍⴷⵉⵜ ⵏ ⵍⵎⵖⵔⵉⴱ 🇲🇦,Glory to Africa ❤️\n#FIFAWorldCup2022 https://...,10,70
3,2022-12-10 23:59:52+00:00,masuma114,"London, England",Why do black England players always get so muc...,0,4
4,2022-12-10 23:59:46+00:00,Mexitly81,"California, USA","No surprise, classless organization. I guess i...",0,0


In [49]:
# Save df to one csv file

FIFA_df.to_csv("FIFA_World_Cup_2022.csv", index = False)

In [130]:
# Read the csv file into dataframe
df = pd.read_csv("FIFA_World_Cup_2022.csv")

In [131]:
df.tail()

,Date,Username,Locations,Content,Retweets,Favorites
617018,2022-11-20 15:07:31+00:00,sataekooklang,NaN,Jungkookie! 😭 \n💜\n#DreamersJungkook \n#Dreame...,10,41
617019,2022-11-20 15:07:31+00:00,Sir_Stevensarp,AngelTown,The tune dey go on though. #FIFAWorldCup,0,1
617020,2022-11-20 15:07:31+00:00,kassimisola,"London, England",I knew the opening ceremony in Qatar will be a...,0,0
617021,2022-11-20 15:07:31+00:00,btslostacc,♡,Jungkook slayed #WorldCup2022,0,0
617022,2022-11-20 15:07:30+00:00,natasyaeffriani,Jakarta,not me crying every time i watch a sports fest...,0,0


## Data Cleaning

### Define 
Check null values

### Code

In [52]:
#Check null values
df.isnull().sum()

Date              0
Username          2
Locations    176454
Content           0
Retweets          0
Favorites         0
dtype: int64

###### The null location is understandable given that not every Twitter user tweets from their location, but I will look into the null value in the username further.

In [53]:
df[df['Username'].isnull()]

,Date,Username,Locations,Content,Retweets,Favorites
452511,2022-11-15 00:22:14+00:00,NaN,"San Antonio, TX",Shirts ordered for #WorldCup2022 https://t.co/...,0,3
528135,2022-11-20 19:06:54+00:00,NaN,"San Antonio, TX",On the road to Doha starting at @SATairport #F...,0,11


### Test

In [54]:
# Drop the null values in the USername column

df.dropna(subset=['Username'], inplace=True)

#Check again
df.isnull().sum()

Date              0
Username          0
Locations    176454
Content           0
Retweets          0
Favorites         0
dtype: int64

In [59]:
# Replace "NaN" values in the location with "Unknown"
df["Locations"].fillna("Unknown", inplace = True)

In [60]:
#Check again
df.isnull().sum()

Date         0
Username     0
Locations    0
Content      0
Retweets     0
Favorites    0
dtype: int64

### Define 
Check for duplicates

### Code

In [61]:
# Check for duolicates
df.duplicated().sum()

15

In [88]:
# Drop the duplicate rows
df.drop_duplicates(subset=None, keep="first", inplace=True)

### Test

In [107]:
#Check again for duplicates
df.duplicated().sum()

0

In [19]:
#Check the description of the dataset
df.describe()

,Retweets,Favorites
count,1.0,1.0
mean,0.0,0.0
std,NaN,NaN
min,0.0,0.0
25%,0.0,0.0
50%,0.0,0.0
75%,0.0,0.0
max,0.0,0.0


In [18]:
# Check the dataset info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype              
---  ------     --------------  -----              
 0   Date       1 non-null      datetime64[ns, UTC]
 1   Username   1 non-null      object             
 2   Locations  1 non-null      object             
 3   Content    1 non-null      object             
 4   Retweets   1 non-null      int64              
 5   Favorites  1 non-null      int64              
dtypes: datetime64[ns, UTC](1), int64(2), object(3)
memory usage: 176.0+ bytes


### Define 
Convert date column to the correct data type

### Code

In [63]:
# Convert the date column to datetime type

df['Date'] = df['Date'].apply(pd.to_datetime)

### Test

In [65]:
# Check the dataset
df['Date'].dtypes

datetime64[ns, UTC]

### Define
I will use Preprocessor library to clean the tweet column and remove - URLs, Hashtags, Mentions, Reserved words (RT, FAV),Emojis and Smileys

In [161]:
# Calling a function to clean the tweet
def preprocess_tweet(df):
    text = df['Content']
    text = p.clean(text)
    return text
df['clean_tweet'] = df.apply(preprocess_tweet, axis=1)

### Test

In [162]:
df.head()

,Date,Username,Locations,Content,Retweets,Favorites,clean_tweet,hashtag
0,2022-12-10 23:59:57+00:00,OSUBlockie,"Columbus, OH",I just earned the 'World Pint (2022)' badge on @untappd! https://t.co/dBY4K1qkbk #football #fifaworldcup #worldcup,0,0,I just earned the 'World Pint (2022)' badge on !,"[football, fifaworldcup, worldcup]"
1,2022-12-10 23:59:54+00:00,standardsport,"London, England",France 🆚 Morocco\n\n📆 Date\n⏰ Kick-off time\n📺 How to watch\n🤕 Team news\n❓Prediction\n💰 Early odds\n\n#FIFAWorldCup semi-final preview\n\n#FRA #MAR\n\n https://t.co/vOYAPqcMSF,0,5,France Morocco Date Kick-off time How to watch Team newsPrediction Early odds semi-final preview,"[FIFAWorldCup, FRA, MAR]"
2,2022-12-10 23:59:52+00:00,MpatrioticM,ⵜⴰⴳⵍⴷⵉⵜ ⵏ ⵍⵎⵖⵔⵉⴱ 🇲🇦,Glory to Africa ❤️\n#FIFAWorldCup2022 https://t.co/3zMNgHvLBe,10,70,Glory to Africa,[FIFAWorldCup2022]
3,2022-12-10 23:59:52+00:00,masuma114,"London, England","Why do black England players always get so much hate after a match when there are other players who missed penalties too? They carry the team and got them this far, stop the racism because if they weren’t in the team your team wouldn’t even get this far.\n#WorldCup2022 #England",0,4,"Why do black England players always get so much hate after a match when there are other players who missed penalties too? They carry the team and got them this far, stop the racism because if they werent in the team your team wouldnt even get this far.","[WorldCup2022, England]"
4,2022-12-10 23:59:46+00:00,Mexitly81,"California, USA","No surprise, classless organization. I guess it won’t matter when @KMbappe wins the golden boot and France 🇫🇷 wins the #WorldCup2022",0,0,"No surprise, classless organization. I guess it wont matter when wins the golden boot and France wins the",[WorldCup2022]


In [159]:
df['hashtag'] = df['Content'].apply(lambda x: re.findall(r'#(\w+)', x))

In [179]:
df.head()

,Date,Username,Locations,Content,Retweets,Favorites,clean_tweet,hashtag
0,2022-12-10 23:59:57+00:00,OSUBlockie,"Columbus, OH",I just earned the 'World Pint (2022)' badge on @untappd! https://t.co/dBY4K1qkbk #football #fifaworldcup #worldcup,0,0,I just earned the 'World Pint (2022)' badge on !,"[football, fifaworldcup, worldcup]"
1,2022-12-10 23:59:54+00:00,standardsport,"London, England",France 🆚 Morocco\n\n📆 Date\n⏰ Kick-off time\n📺 How to watch\n🤕 Team news\n❓Prediction\n💰 Early odds\n\n#FIFAWorldCup semi-final preview\n\n#FRA #MAR\n\n https://t.co/vOYAPqcMSF,0,5,France Morocco Date Kick-off time How to watch Team newsPrediction Early odds semi-final preview,"[FIFAWorldCup, FRA, MAR]"
2,2022-12-10 23:59:52+00:00,MpatrioticM,ⵜⴰⴳⵍⴷⵉⵜ ⵏ ⵍⵎⵖⵔⵉⴱ 🇲🇦,Glory to Africa ❤️\n#FIFAWorldCup2022 https://t.co/3zMNgHvLBe,10,70,Glory to Africa,[FIFAWorldCup2022]
3,2022-12-10 23:59:52+00:00,masuma114,"London, England","Why do black England players always get so much hate after a match when there are other players who missed penalties too? They carry the team and got them this far, stop the racism because if they weren’t in the team your team wouldn’t even get this far.\n#WorldCup2022 #England",0,4,"Why do black England players always get so much hate after a match when there are other players who missed penalties too? They carry the team and got them this far, stop the racism because if they werent in the team your team wouldnt even get this far.","[WorldCup2022, England]"
4,2022-12-10 23:59:46+00:00,Mexitly81,"California, USA","No surprise, classless organization. I guess it won’t matter when @KMbappe wins the golden boot and France 🇫🇷 wins the #WorldCup2022",0,0,"No surprise, classless organization. I guess it wont matter when wins the golden boot and France wins the",[WorldCup2022]


# Need to remove punctuation, number and convert to lower case

In [66]:
# Save cleaned dataframe
df.to_pickle("cleaned.csv")

In [67]:
# Load
df1 = pd.read_pickle("cleaned.csv")

In [68]:
# Check
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 617021 entries, 0 to 617022
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype              
---  ------     --------------   -----              
 0   Date       617021 non-null  datetime64[ns, UTC]
 1   Username   617021 non-null  object             
 2   Locations  617021 non-null  object             
 3   Content    617021 non-null  object             
 4   Retweets   617021 non-null  int64              
 5   Favorites  617021 non-null  int64              
dtypes: datetime64[ns, UTC](1), int64(2), object(3)
memory usage: 33.0+ MB


In [127]:
df1.describe()

,Retweets,Favorites
count,617021.000000,617021.000000
mean,5.651132,33.051010
std,457.314753,1481.927339
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,1.000000
75%,0.000000,2.000000
max,296698.000000,793393.000000


In [126]:
df1.Locations.unique()

array(['Columbus, OH', 'London, England', 'ⵜⴰⴳⵍⴷⵉⵜ ⵏ ⵍⵎⵖⵔⵉⴱ 🇲🇦', ...,
       '🇰🇪🇮🇪', 'Come into my world', 'Hucknall, Nottinghamshire'],
      dtype=object)